## 导入依赖库

这里导入相关的依赖库，主要为mindnlp， transformers，peft等常用库。transformers中定义了典型的transformer结构的模型，peft中实现了LoRA微调流程。

In [ ]:
import mindnlp
import mindspore
from mindnlp import core
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForSeq2Seq, TrainingArguments, Trainer, GenerationConfig
from peft import LoraConfig, TaskType, get_peft_model, PeftModel

In [5]:
import pandas as pd
import json
from datasets import Dataset

ImportError: The pyarrow installation is not built with support for the Parquet file format (DLL load failed while importing _parquet: 找不到指定的模块。)

## 下载并格式转换数据

通过pd.read_json() 读取JSON文件到Pandas DataFrame，Dataset.from_pandas() 将DataFrame转换为Hugging Face的Dataset对象。	

In [2]:
# 数据加载并进行格式转换
df = pd.read_json('d:\\extract-dialogue\\output\\converted_sunwukong_lines_up.jsonl', lines=True)
ds = Dataset.from_pandas(df)
ds[:3]

NameError: name 'Dataset' is not defined

## 实例化分词器

实例化DeepSeek-R1-Distill-Qwen-1.5B的分词器，用于下一步的数据预处理。

In [ ]:
# 实例化tokenizer
tokenizer = AutoTokenizer.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', use_fast=False, trust_remote_code=True)
tokenizer

## 数据预处理

process_func函数的作用是将用户的指令（instruction）、输入内容（input）和模型的期望回复（output）格式化为适合语言模型训练的输入数据。主要包含以下处理：
1. 对话模板构建：按system：[指令]，User: [输入] 和 Assistant: [回复] 的格式组织对话
2. 分词与编码：用步骤3中的分词器将文本转换为模型可理解的数字序列
3. 长度控制：截断超长内容
4. 生成掩码和标签：标识有效内容和需要模型学习的部分


In [ ]:
def process_func(example):
    MAX_LENGTH = 384    # Llama分词器会将一个中文字切分为多个token，因此需要放开一些最大长度，保证数据的完整性
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(f"<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛<|im_end|>\n<|im_start|>user\n{example['instruction'] + example['input']}<|im_end|>\n<|im_start|>assistant\n", add_special_tokens=False)  # add_special_tokens 不在开头加 special_tokens
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = instruction["attention_mask"] + response["attention_mask"] + [1]  # 因为eos token咱们也是要关注的所以 补充为1
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]  
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels
    }


In [ ]:
tokenized_id = ds.map(process_func, remove_columns=ds.column_names)
tokenized_id

In [ ]:
tokenizer.decode(tokenized_id[0]['input_ids'])

## 模型加载

此步骤是在配置Lora参数之前，需要先加载模型，本实验用DeepSeek-R1-Distill-Qwen-1.5B作为基础模型。加载时需要从镜像站下载模型的权重文件，耗时较长，请耐心等待。

In [ ]:
# 加载基础模型
model = AutoModelForCausalLM.from_pretrained('deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B', ms_dtype=mindspore.bfloat16, device_map=0)

# 开启梯度检查点时，要执行该方法
model.enable_input_require_grads()

## 微调前推理

进行模型推理查看效果

In [ ]:
# host to device
model = model.npu()

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "现在你要扮演皇帝身边的女人--甄嬛"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="ms",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with core.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


## 配置LoRA参数

LoRA（Low-Rank Adaptation） 是一种高效微调大模型的技术，核心思想是冻结原始模型参数，通过向特定层注入低秩矩阵来实现参数更新，从而节省计算资源和内存。
其中一些重要参数的意义如下：r用于控制低秩矩阵的维度、lora_alpha用于表示缩放因子、
target_modules用于指定需要添加LoRA的层。


In [ ]:
# 配置LoRA
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)
config


对比前后模型变化

In [ ]:
print("Model without LoRA:\n",model)
# 根据上述的lora配置，为模型添加lora部分
model = get_peft_model(model, config)
print('='*50)
print("Model with LoRA:\n",model)
# 输出打印需要训练的参数比例
model.print_trainable_parameters()

## 模型训练

本步骤中将完成训练中的参数配置，最终实例化Trainer启动模型的训练进程。
首先，对训练中的参数进行配置，包括num_train_epochs训练轮次、learning_rate学习率、per_device_train_batch_size每批次的数据条数大小等。注意，训练中间过程输出的权重文件将会在output_dir参数指定的目录下。


In [ ]:
# 定义训练超参数
args = TrainingArguments(
    output_dir="./output_1.5bf/Qwen2.5_instruct_lora",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=5,
    logging_steps=10,
    num_train_epochs=3,
    save_steps=100, 
    learning_rate=1e-4,
    save_on_each_node=True,
)


最后，实例化Trainer类，将上述定义的参数传入，启动训练进程。

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_id,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

trainer.train()


## 微调后推理


In [ ]:
mode_path = 'deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B'
lora_path = './output_1.5bf/Qwen2.5_instruct_lora/checkpoint-561' # 这里改称你的 lora 输出对应 checkpoint 地址

# 加载tokenizer
tokenizer = AutoTokenizer.from_pretrained(mode_path, trust_remote_code=True)

# 加载模型
model = AutoModelForCausalLM.from_pretrained(mode_path, ms_dtype=mindspore.bfloat16, trust_remote_code=True).eval()

# 加载lora权重
model = PeftModel.from_pretrained(model, model_id=lora_path)

# host to device
model = model.npu()

prompt = "你是谁？"
inputs = tokenizer.apply_chat_template([{"role": "system", "content": "现在你要扮演皇帝身边的女人--甄嬛"},{"role": "user", "content": prompt}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="ms",
                                       return_dict=True
                                       ).to('cuda')


gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with core.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
